# Dissect Notebook

This notebook is created to accompany the DISSECT article:

*DisPerSE-based Segmentation and Exploration of Cells and Tissues, in highly remodeled 3D epithelia* 

This notebook uses Panel (https://panel.holoviz.org/index.html) a powerfull tool to bring interactivity on  jupyter notebook

The Notebook is divided into five more or less dependent parts


1 - **Segmentation**

    This segmentation part includes two independent functions. The user must select     
	- NDskl file
	- Tiff file
    
     The first function "binontiff" allows user to display the raw images of :
		- Original Tiff image
		- NDskl image
		- Superposition of the two Raws
        
		execution time: ~ 4 minutes
        
     The second function "cleantask" allows user to display images of:
		- Clean Skeleton + vertices
		- Junctions ID
		- Faces ID
        
		execution time: ~ 80 minutes the first time and ~ 5 minutes after the first time 


2 - **Morphological Analysis**

      This part is dependent on the Segmentation part it has two functions
      The user chooses the parameters he wishes to display: the Area, the Perimeter, the neighbors and the anisotropy
      The principal function is "morphoana" allows user to display:
	    - A table of selected values
	    - Images distribution of selected parameters
	   execution time: ~ 45 minutes the first time and ~ 8 minutes after the first time
	  The second function "juncana"  allows user to display plane orientation of junctions
		- XY
		- XZ
		- YZ
        execution time: ~ 240 minutes the first time and 15min after the first time 

3 - **Apical surface intensity**

       This part is dependent on the first part (Segmentation)
       The user selects the image of the second signal and chooses a label
       execution time: ~ 90 minutes

4 - **Network analysis**

    This part is independent of all other parts
    User must select for this part
	- NDskl file of network
	- Tiff file of network
    The main function is "networktask"
    This function allows the user to display the images of:
       - Branch orientation
	   - Branch length
	   - Branch tortuosity
	   - Branch mean
	   - Branch centrality
       execution time: ~ 120 minutes the first time, ~ 15 minutes next time

5 - **Cross analysis**

     This part is dependent on parts 1 and 4 (Segmentation and Networks)
     The main function "crossanal" allows the user to display the images of:
     - max_degree
     - nb_node_pcell
     - connection
     - sum_degree
     - sum_between
     - sum_closen
     - mean_closen
     - mean_between
     - mean_degre
     
     execution time: ~ 90 minutes the first time, ~ 15 minutes next time
     

## Packages import

In [1]:
import numpy as np
from scipy import ndimage
from tifffile import (imsave, imread)
import panel as pn
import pandas as pd
import panel.widgets as pnw
pn.config.sizing_mode='stretch_width'
pn.extension('vtk')

### First step of Panel tools construction

In [2]:
master_dir = '/home/kamgoue/Rundiss'

In [3]:
app_bar = pn.Row(
    pn.pane.PNG("Logo_v2.png", height=50, sizing_mode="fixed", align="center"),
    pn.pane.Markdown("# DisPerSE Segmentation and Exploration of Cells and Tissues", style={"color": "white"}, width=500, sizing_mode="fixed", margin=(10,5,10,15)), 
    pn.Spacer(),
    pn.pane.PNG("https://panel.holoviz.org/_static/logo_horizontal.png", height=50, sizing_mode="fixed", align="center"),
    background="black",)
selfil = pn.widgets.FileSelector(master_dir)
voxelX = pn.widgets.input.TextInput(name='', placeholder='X Voxel size (µm)')
voxelY = pn.widgets.input.TextInput(name='', placeholder='Y Voxel size (µm)')
voxelZ = pn.widgets.input.TextInput(name='', placeholder='Z Voxel size (µm)')
thick = pn.widgets.input.TextInput(name='', placeholder='Thickness (µm)')
juncdil = pn.widgets.EditableIntSlider(name='Junction Dilation', start=1, end=5, step=1, value=3)
voxelX2 = pn.widgets.input.TextInput(name='', placeholder='X Voxel size (µm)')
voxelY2 = pn.widgets.input.TextInput(name='', placeholder='Y Voxel size (µm)')
voxelZ2 = pn.widgets.input.TextInput(name='', placeholder='Z Voxel size (µm)')

textindic0 = pn.pane.Alert('## Alert\nThis is a warning!')
textindic = pn.pane.Alert('## Alert\nThis is a warning!')

buttonsrawim = pn.widgets.Button(name='Raw Image', button_type='warning')
buttonraskel = pn.widgets.Button(name='Raw Skeleton', button_type='danger')
buttonimskel = pn.widgets.Button(name='Skeleton + Image', button_type='success')
buttonskelontif = pn.widgets.Button(name='Skel on Image', button_type='primary')

buttonstartclean = pn.widgets.Button(name='Start Cleaning', button_type='primary')
buttonskelvert = pn.widgets.Button(name='\uD83D\uDCBE Skel & Vertices', button_type='warning')
buttonjuncid = pn.widgets.Button(name='\uD83D\uDCBE Junction ID', button_type='danger')
buttoncellid = pn.widgets.Button(name='\uD83D\uDCBE Cell ID', button_type='success')

buttonjorient = pn.widgets.Button(name='\uD83D\uDCBE Branch Orientation', button_type='warning')
buttonjlength = pn.widgets.Button(name='\uD83D\uDCBE Branch Length', button_type='danger')
buttonnetwork = pn.widgets.Button(name='\uD83D\uDCBE Network Image', button_type='success')

buttonjtortuo = pn.widgets.Button(name='\uD83D\uDCBE Branch Tortuosity', button_type='warning')
buttonjmean = pn.widgets.Button(name='\uD83D\uDCBE Branch Mean', button_type='danger')
buttonjcentr = pn.widgets.Button(name='\uD83D\uDCBE Branch Centrality', button_type='success')

buttonarea = pn.widgets.Button(name='\uD83D\uDCBE Area', button_type='warning')
buttonperim = pn.widgets.Button(name='\uD83D\uDCBE Perimeter', button_type='danger')
buttonneigb = pn.widgets.Button(name='\uD83D\uDCBE Neighbor', button_type='success')
buttonaniso = pn.widgets.Button(name='\uD83D\uDCBE Anisotropy', width=50, button_type='warning')
buttonanisov = pn.widgets.Button(name='', button_type='warning')

multi_choice = pn.widgets.MultiChoice(name='Parameters', value=['Area', 'Anisotropy'],
    options=['Area', 'Perimeter', 'Anisotropy', 'Neighbors'])
buttonmorphoanaly = pn.widgets.Button(name='Morpho Analyze', button_type='primary')
buttonjuncanaly = pn.widgets.Button(name='Junction Analyze', button_type='primary')
#buttonstartanaly.on_click(b2)
selfil2 = pn.widgets.FileSelector(master_dir)
textindic2 = pn.pane.Alert('## Alert\nThis is a warning!')
buttonstartnet = pn.widgets.Button(name='Start Network', button_type='primary')
selfil3 = pn.widgets.FileSelector('/home/kamgoue')
textindic3 = pn.pane.Alert('## Alert\nGive other Tiff Label', alert_type="success")
tiffother = pn.widgets.input.TextInput(name='', placeholder='Label')
buttontiffother = pn.widgets.Button(name='Apical Surface Intensity', button_type='primary')

btn_max_degree_image = pn.widgets.Button(name='\uD83D\uDCBE Max Degree', button_type='warning')
btn_nb_node_pcell = pn.widgets.Button(name='\uD83D\uDCBE Number of Node', button_type='danger')
btn_connection = pn.widgets.Button(name='\uD83D\uDCBE Connection', button_type='success')
btn_sum_degree = pn.widgets.Button(name='\uD83D\uDCBE Sum Degree', button_type='warning')
btn_sum_between = pn.widgets.Button(name='\uD83D\uDCBE Sum Betweenness', button_type='danger')
btn_sum_closen = pn.widgets.Button(name='\uD83D\uDCBE Sum Closenness', button_type='success')
btn_mean_degre = pn.widgets.Button(name='\uD83D\uDCBE Mean Degree', button_type='warning')
btn_mean_between = pn.widgets.Button(name='\uD83D\uDCBE Mean Betweenness', button_type='danger')
btn_mean_closen = pn.widgets.Button(name='\uD83D\uDCBE Mean Closenness', button_type='success')

textindic4 = pn.pane.Alert('## Alert\n ', alert_type="danger")
buttoncross = pn.widgets.Button(name='Cross Analysis', button_type='primary')

## Segmentation functions

In [4]:
def binontiff(event):
    import os
    from basemodule import (Cleaning,Binary_image,Skel0)
    import pickle
    
    if (len(selfil.value)>=2):
        if selfil.value[0][-5:]=='NDskl':
            fileskel = selfil.value[0]
            originalfil = selfil.value[1]
        else:
            originalfil = selfil.value[0]
            fileskel = selfil.value[1]
        textindic0.object = '## Alert\n' + ' Nice we proceed'
        textindic0.alert_type = 'success'
        voxelX0,voxelY0,voxelZ0,thick0,juncdil0 = voxelX.value,voxelY.value,voxelZ.value,thick.value,juncdil.value
        workingdir = os.path.dirname(fileskel)
        file = os.path.basename(fileskel)
        cud, newd = os.getcwd(), file[:-5]
        skel0 = Skel0(workingdir,file)
        image_skel0 = skel0.create_binary_image()
            
        image_skel0 = image_skel0.astype(float)
            
        im = imread(originalfil)
        im = im.astype(float)
        Maxim = 5*im.max() 
        faceid = Maxim*image_skel0 + im
            
        os.chdir(cud)
        textindic0.object = '## Alert\n' + ' Beginning plot'
        plt0=pn.pane.VTKVolume(im, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
        plt1=pn.pane.VTKVolume(image_skel0, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
        plt2=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
                
    else:
        textindic0.object = '## Alert\n' + ' You need skeleton file and original file'
        textindic0.alert_type = 'danger'
        faceid = np.zeros((10,10,10))
        
        plt0=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt1, plt2 = plt0, plt0
    
    plt0.jslink(plt1, camera='camera', bidirectional=True)
    plt1.jslink(plt2, camera='camera', bidirectional=True)
    plt2.jslink(plt0, camera='camera', bidirectional=True)
    
    return pn.Row(plt0, plt1, plt2)

In [5]:
def cleantask(event):
    import os
    from basemodule import (Cleaning, Images_vert, Vertex_image, Images_junc)
    from basemodule import (Binary_image, Images_face, Face_Edge_Vert, Face_image)
    import pickle
    
    if (len(selfil.value)>=2):
        if selfil.value[0][-5:]=='NDskl':
            fileskel = selfil.value[0]
            originalfil = selfil.value[1]
        else:
            originalfil = selfil.value[0]
            fileskel = selfil.value[1]
        textindic.object = '## Alert\n' + ' Nice we proceed'
        textindic.alert_type = 'success'
        voxelX0,voxelY0,voxelZ0,thick0,juncdil0 = voxelX.value,voxelY.value,voxelZ.value,thick.value,juncdil.value
        workingdir = os.path.dirname(fileskel)
        file = os.path.basename(fileskel)
        cud, newd = os.getcwd(), file[:-5]
        os.chdir(workingdir)
        if os.path.isdir('Arrays_' + newd):
            os.chdir('Arrays_' + newd)
            with open('segmentation.pkl', 'rb') as inp:
                segmentation = pickle.load(inp)

            textindic.object = '## Alert\n' + ' Cleaning already done'
            if os.path.isfile('skelvert.npz'):
                bb1 = np.load('skelvert.npz')
                skelbin = bb1['binary']
                skelbin = np.array(skelbin)
                skelbin = skelbin.astype(float)
            
                bb2 = bb1['vertx']
                vertx = np.array(bb2)
                vertx = 5*vertx.astype(float)
                im = imread(originalfil)
                im = im.astype(float)
                #Maxim = im.max() 
                faceid = skelbin + vertx
            
                bb03 = bb1['facej']
                facej = np.array(bb03)
                facej = facej.astype(float)
            
                bb3 = bb1['face_image']
                face = np.array(bb3)
                face = face.astype(float)
            else:
                skelet = Cleaning(workingdir,fileskel)
                binary = Binary_image(skelet)
                segmentation = Face_Edge_Vert(skelet,workingdir,originalfil,float(voxelX0), float(voxelY0),float(voxelZ0))
                with open('segmentation4cross.pkl', 'wb') as outp:
                    pickle.dump(segmentation, outp, pickle.HIGHEST_PROTOCOL)
                with open('segmentation.pkl', 'wb') as outp:
                    pickle.dump(segmentation, outp, pickle.HIGHEST_PROTOCOL)
                face_image = Images_face(segmentation, workingdir, originalfil, float(thick0))                
                #face_image = Face_image(workingdir,originalfil,segmentation,float(thick0),int(juncdil0),float(voxelX0),float(voxelY0),float(voxelZ0))
                
                vertx = Vertex_image(workingdir,originalfil,segmentation.vert_df)
                facej = Images_junc(segmentation,workingdir,originalfil)
                np.savez_compressed('skelvert.npz', binary=binary, face_image=face_image, vertx=vertx, facej=facej)
                skelbin = binary.astype(float)
                vertx = 5*vertx.astype(float)
                im = imread(originalfil)
                im = im.astype(float)
                #Maxim = im.max() 
                faceid = skelbin + vertx
                face = face_image.astype(float)
                facej = facej.astype(float)
                    
            os.chdir(cud)
            textindic.object = '## Alert\n' + ' Beginning plot'
            plt0=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt1=pn.pane.VTKVolume(facej, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt2=pn.pane.VTKVolume(face, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt0.jslink(plt1, camera='camera', bidirectional=True)
            plt1.jslink(plt2, camera='camera', bidirectional=True)
            plt2.jslink(plt0, camera='camera', bidirectional=True)
            
        else:
            skelet = Cleaning(workingdir,fileskel)
            segmentation = Face_Edge_Vert(skelet,workingdir,originalfil,float(voxelX0), float(voxelY0),float(voxelZ0))
            
            binary = Binary_image(skelet)
            face_image = Images_face(segmentation, workingdir, originalfil, float(thick0)) 
            vertx = Vertex_image(workingdir,originalfil,segmentation.vert_df)
            facej = Images_junc(segmentation,workingdir,originalfil)
            os.chdir(workingdir)
            os.mkdir('Arrays_' + newd)
            os.chdir('Arrays_' + newd)
            np.savez_compressed('skelvert.npz', binary=binary, face_image=face_image, vertx=vertx, facej=facej)
            with open('segmentation.pkl', 'wb') as outp:
                pickle.dump(segmentation, outp, pickle.HIGHEST_PROTOCOL)
            with open('segmentation4cross.pkl', 'wb') as outp:
                    pickle.dump(segmentation, outp, pickle.HIGHEST_PROTOCOL)
            os.chdir(cud)
            textindic.object = '## Alert\n' + ' Cleaning Done'
            faceid = binary.astype(float) + 5*vertx.astype(float)
            face = np.array(face_image)
            face = face.astype(float)
            facej = np.array(facej)
            facej = facej.astype(float)
            
        plt0=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=300, height=600, 
                               spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                               edge_gradient=0, sampling=0)
        plt1=pn.pane.VTKVolume(facej, sizing_mode='stretch_width', width=300, height=600, 
                               spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                               edge_gradient=0, sampling=0)
        plt2=pn.pane.VTKVolume(face, sizing_mode='stretch_width', width=300, height=600, 
                               spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                               edge_gradient=0, sampling=0)
        plt0.jslink(plt1, camera='camera', bidirectional=True)
        plt1.jslink(plt2, camera='camera', bidirectional=True)
        plt2.jslink(plt0, camera='camera', bidirectional=True)
    else:
        textindic.object = '## Alert\n' + ' You need skeleton file and original file'
        textindic.alert_type = 'danger'
        faceid = np.zeros((10,10,10))
        
        plt0=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt1,plt2 = plt0,plt0
           
    
    return pn.Row(plt0, plt1, plt2)
#buttonstartclean.on_click(cleantask)

## Networt function

In [6]:
def networktask(event):
    import os
    from basemodule import (Skel_Junc, Junc_len, Image_myo)
    from basemodule import (Junc_tortuo, Junc_mean, Centrality)
    import pickle
    
    voxelX0,voxelY0,voxelZ0 = voxelX2.value,voxelY2.value,voxelZ2.value
    
    if (len(selfil.value)>=2):
        if selfil.value[0][-5:]=='NDskl':
            fileskel = selfil.value[0]
            originalfil = selfil.value[1]
        else:
            originalfil = selfil.value[0]
            fileskel = selfil.value[1]
        workingdir = os.path.dirname(fileskel)
        file = os.path.basename(fileskel)
        cud, newd = os.getcwd(), file[:-5]
    else:
        faceid = np.zeros((10,10,10))
        plt3=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=225, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt1,plt2 = plt3,plt3
        return pn.Column(pn.Row(plt1, plt2, plt3),)
    
    if (len(selfil2.value)>=2):
        if selfil2.value[0][-5:]=='NDskl':
            fileskel2 = selfil2.value[0]
            originalfil2 = selfil2.value[1]
        else:
            originalfil2 = selfil2.value[0]
            fileskel2 = selfil2.value[1]
        textindic2.object = '## Alert\n' + ' Nice we proceed'
        textindic2.alert_type = 'success'
        workingdir2 = os.path.dirname(fileskel2)
        file2 = os.path.basename(fileskel2)
        cud, newd2 = os.getcwd(), file2[:-5]
        os.chdir(workingdir2)
        if os.path.isdir('Arrays_' + newd2):
            os.chdir('Arrays_' + newd2)
            textindic2.object = '## Alert\n' + ' Network already done  ...wait'
            if os.path.isfile('netw.npz'):
                bb1 = np.load('netw.npz')
                jorient = np.array(bb1['jorient']).astype(float)
                jlength = np.array(bb1['jlength']).astype(float)
                jtortuo = np.array(bb1['jtortuo']).astype(float)
                jmean = np.array(bb1['jmean']).astype(float)
                jcentr = np.array(bb1['jcentr']).astype(float)
            else:
                image_myo_ap_dil, skel_myo_ap = Image_myo(workingdir2, originalfil2, fileskel2)
                jorient, df_junc_myo = Skel_Junc(skel_myo_ap, workingdir2,originalfil2, float(voxelX0), 
                                             float(voxelY0), float(voxelZ0))
                jlength = Junc_len(workingdir2,originalfil2,df_junc_myo)
                jtortuo = Junc_tortuo(workingdir2,originalfil2,df_junc_myo)
                jmean = Junc_mean(workingdir2, originalfil2, df_junc_myo)
                jcentr = Centrality(workingdir2,originalfil2,df_junc_myo,skel_myo_ap)
                np.savez_compressed('netw.npz', jorient=jorient, jlength=jlength, 
                                    jtortuo=jtortuo, jmean=jmean, jcentr = jcentr)
                jorient, jlength, jtortuo = jorient.astype(float), jlength.astype(float), jtortuo.astype(float)
                jmean, jcentr = jmean.astype(float), jcentr.astype(float)
            
            im2 = imread(originalfil2)
            im2 = im2.astype(float)
            
            jce = jcentr.astype(float)
            mmax = jce.max()
            os.chdir(workingdir)
            os.chdir('Arrays_' + newd)
            bb1 = np.load('skelvert.npz')
            skelbin = bb1['binary']
            skelbin = np.array(skelbin)
            skelbin = skelbin.astype(float)
            
            jcentr = jce + (mmax + 1)*skelbin
            
            os.chdir(cud)
            textindic.object = '## Alert\n' + ' Beginning plot'
            plt0=pn.pane.VTKVolume(im2, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt1=pn.pane.VTKVolume(jorient, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt2=pn.pane.VTKVolume(jlength, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt3=pn.pane.VTKVolume(jtortuo, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt4=pn.pane.VTKVolume(jmean, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt5=pn.pane.VTKVolume(jcentr, sizing_mode='stretch_width', width=300, height=600, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt0.jslink(plt1, camera='camera', bidirectional=True)
            plt1.jslink(plt2, camera='camera', bidirectional=True)
            plt2.jslink(plt3, camera='camera', bidirectional=True)
            plt3.jslink(plt4, camera='camera', bidirectional=True)
            plt4.jslink(plt5, camera='camera', bidirectional=True)
            plt5.jslink(plt0, camera='camera', bidirectional=True)

        else:
            os.mkdir('Arrays_' + newd2)
            os.chdir('Arrays_' + newd2)
            image_myo_ap_dil, skel_myo_ap = Image_myo(workingdir2, originalfil2, fileskel2)
            jorient, df_junc_myo = Skel_Junc(skel_myo_ap, workingdir2,originalfil2, float(voxelX0), 
                                             float(voxelY0), float(voxelZ0))
            jlength = Junc_len(workingdir2,originalfil2,df_junc_myo)
            jtortuo = Junc_tortuo(workingdir2,originalfil2,df_junc_myo)
            jmean = Junc_mean(workingdir2, originalfil2, df_junc_myo)
            jcentr = Centrality(workingdir2,originalfil2,df_junc_myo,skel_myo_ap)
            np.savez_compressed('netw.npz', jorient=jorient, jlength=jlength,jtortuo=jtortuo, jmean=jmean, jcentr = jcentr)
            jorient, jlength, jtortuo = jorient.astype(float), jlength.astype(float), jtortuo.astype(float)
            jmean, jcentr = jmean.astype(float), jcentr.astype(float)
            im2 = imread(originalfil2)
            im2 = im2.astype(float)
            plt0=pn.pane.VTKVolume(im2, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
            plt1=pn.pane.VTKVolume(jorient, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
            plt2=pn.pane.VTKVolume(jlength, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
            plt3=pn.pane.VTKVolume(jtortuo, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
            plt4=pn.pane.VTKVolume(jmean, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
            plt5=pn.pane.VTKVolume(jcentr, sizing_mode='stretch_width', width=300, height=600, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
            plt0.jslink(plt1, camera='camera', bidirectional=True)
            plt1.jslink(plt2, camera='camera', bidirectional=True)
            plt2.jslink(plt3, camera='camera', bidirectional=True)
            plt3.jslink(plt4, camera='camera', bidirectional=True)
            plt4.jslink(plt5, camera='camera', bidirectional=True)
            plt5.jslink(plt1, camera='camera', bidirectional=True)
        return pn.Column(pn.Row(buttonnetwork,buttonjorient,buttonjlength),
                         pn.Row(plt0, plt1, plt2),
                         pn.Row(plt3, plt4, plt5),
                         pn.Row(buttonjtortuo,buttonjmean,buttonjcentr)
                        )
    else:
        textindic2.object = '## Alert\n' + ' You need skeleton file and original file'
        textindic2.alert_type = 'danger'
        faceid2 = np.zeros((10,10,10))
        
        plt0=pn.pane.VTKVolume(faceid2, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt1,plt2,plt3,plt4,plt5 = plt0,plt0,plt0,plt0, plt0
        return pn.Column(pn.Row(buttonnetwork,buttonjorient,buttonjlength),
                         pn.Row(plt0, plt1, plt2),
                         pn.Row(plt3, plt4, plt5),
                         pn.Row(buttonjtortuo,buttonjmean,buttonjcentr)
                        )
        


## Morphological analysis function

In [7]:
def morphoana(event):
    import os
    from basemodule import (AreaFace, PerimeterFace, AnisoFace, NeighFace, Morpho_analy)
    from basemodule import (Orientationx, Orientationy, Orientationz)
    import pickle
    
    if (len(selfil.value)>=2):
        if selfil.value[0][-5:]=='NDskl':
            fileskel = selfil.value[0]
            originalfil = selfil.value[1]
        else:
            originalfil = selfil.value[0]
            fileskel = selfil.value[1]
        textindic.object = '## Alert\n' + ' Nice we proceed'
        textindic.alert_type = 'success'
        voxelX0,voxelY0,voxelZ0,thick0,juncdil0 = voxelX.value,voxelY.value,voxelZ.value,thick.value,juncdil.value
        workingdir = os.path.dirname(fileskel)
        file = os.path.basename(fileskel)
        cud, newd = os.getcwd(), file[:-5]
        os.chdir(workingdir)
        os.chdir('Arrays_' + newd)
        with open('segmentation.pkl', 'rb') as inp:
            segmentation = pickle.load(inp)
        os.chdir(cud)
        os.chdir(workingdir)
        os.chdir('Arrays_' + newd)
        if os.path.isfile('morpho.npz'):
            bb1 = np.load('morpho.npz')
            area = bb1['area']
            perimeter = bb1['perimeter']
            neig = bb1['neig']
            aniso = bb1['aniso']
        else:
            rbtar, rbtpr, rbtnb, rbtan, rbtjo = True, True, True, True, False
            segmentation = Morpho_analy(segmentation,rbtar,rbtpr,rbtnb,rbtan,rbtjo)
            with open('segmentation.pkl', 'wb') as outp:
                pickle.dump(segmentation, outp, pickle.HIGHEST_PROTOCOL)
            with open('segmentation2.pkl', 'wb') as outp:
                pickle.dump(segmentation, outp, pickle.HIGHEST_PROTOCOL)
            area = AreaFace(segmentation)
            perimeter = PerimeterFace(segmentation)
            neig = NeighFace(segmentation)
            aniso = AnisoFace(segmentation)
            np.savez_compressed('morpho.npz', area=area, perimeter=perimeter, neig=neig, aniso=aniso)
            
        Lp = []
        rbtar, rbtpr, rbtnb, rbtan, rbtjo = False, False, False, False, False
        if 'Area' in multi_choice.value:
            rbtar = True
            Lp.append('area')
        if 'Perimeter' in multi_choice.value:
            rbtpr = True
            Lp.append('perimeter')
        if 'Neighbors' in multi_choice.value:
            rbtnb = True
            Lp.append('nb_neighbor')
        if 'Anisotropy' in multi_choice.value:
            rbtan = True
            Lp.append('aniso')

        textindic.object = '## Alert\n' + ' Analysis is done'
        with open('segmentation2.pkl', 'rb') as inp:
            segmentation = pickle.load(inp)
        tablo = segmentation.face_df
        tablo2 = tablo[Lp]
        tablo2.to_csv(os.path.join(workingdir, 'facetablo_df.csv'))
        if rbtar == True:
            area = np.array(area)
            area.astype(float)
            os.chdir(cud)
        else:
            area = np.zeros((10,10,10))
        if rbtpr == True:
            perimeter = np.array(perimeter)
            perimeter.astype(float)
            os.chdir(cud)
        else:
            perimeter = np.zeros((10,10,10))
        if rbtnb == True:
            neig = np.array(neig)
            neig.astype(float)
            os.chdir(cud)
        else:
            neig = np.zeros((10,10,10))
        if rbtan == True:
            aniso = np.array(aniso)
            ani = aniso.astype(float)
            mmax = ani.max()
            os.chdir(workingdir)
            os.chdir('Arrays_' + newd)
            bb1 = np.load('skelvert.npz')
            skelbin = bb1['binary']
            skelbin = np.array(skelbin)
            skelbin = skelbin.astype(float)
            
            aniso = ani + (mmax + 1)*skelbin
            os.chdir(cud)
        else:
            aniso = np.zeros((10,10,10))

            
        plt3=pn.pane.VTKVolume(area, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt4=pn.pane.VTKVolume(perimeter, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt5=pn.pane.VTKVolume(neig, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt6=pn.pane.VTKVolume(aniso, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt7=pn.pane.VTKVolume(np.zeros((10,10,10)), sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt8=plt7
        plt3.jslink(plt4, camera='camera', bidirectional=True)
        plt4.jslink(plt5, camera='camera', bidirectional=True)
        plt5.jslink(plt6, camera='camera', bidirectional=True)
        plt6.jslink(plt3, camera='camera', bidirectional=True)

        return pn.Column(pn.widgets.DataFrame(tablo2, name='DataFrame'),
                         pn.Row(buttonarea, buttonperim, buttonneigb),
                         pn.Row(plt3, plt4, plt5),
                         pn.Row(buttonaniso,buttonanisov,buttonanisov),
                         pn.Row(plt6,plt7,plt8)
                        )
    else:
        textindic.object = '## Alert\n' + ' You need skeleton file and original file'
        textindic.alert_type = 'danger'
        faceid = np.zeros((10,10,10))
        data = []
        tablo2 = pd.DataFrame(data, columns=['A', 'B', 'C'])
        plt3=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=225, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt4,plt5,plt6,plt7,plt8 = plt3,plt3,plt3,plt3,plt3
        return pn.Column(pn.widgets.DataFrame(tablo2, name='DataFrame'),
                         pn.Row(buttonarea, buttonperim, buttonneigb),
                         pn.Row(plt3, plt4, plt5),
                         pn.Row(buttonaniso,buttonanisov,buttonanisov),
                         pn.Row(plt6,plt7,plt8)
                        )
    


In [8]:
def juncana(event):
    import os
    from basemodule import (OrientationXY, OrientationYZ, OrientationXZ, Morpho_analy, LengthJunc)
    import pickle
    
    if (len(selfil.value)>=2):
        if selfil.value[0][-5:]=='NDskl':
            fileskel = selfil.value[0]
            originalfil = selfil.value[1]
        else:
            originalfil = selfil.value[0]
            fileskel = selfil.value[1]
        textindic.object = '## Alert\n' + ' Nice we proceed'
        textindic.alert_type = 'success'
        voxelX0,voxelY0,voxelZ0,thick0,juncdil0 = voxelX.value,voxelY.value,voxelZ.value,thick.value,juncdil.value
        workingdir = os.path.dirname(fileskel)
        file = os.path.basename(fileskel)
        cud, newd = os.getcwd(), file[:-5]
        os.chdir(workingdir)
        os.chdir('Arrays_' + newd)
        with open('segmentation.pkl', 'rb') as inp:
            segmentation = pickle.load(inp)
        os.chdir(cud)
        rbtar, rbtpr, rbtnb, rbtan, rbtjo = False, False, False, False, True
        segmentation = Morpho_analy(segmentation,rbtar,rbtpr,rbtnb,rbtan,rbtjo)
        textindic.object = '## Alert\n' + ' Analysis is done'
        tablo_edg = segmentation.edge_df
        tablo_edg.to_csv(os.path.join(workingdir, 'edgetablo00_df.csv'))

        os.chdir(workingdir)
        os.chdir('Arrays_' + newd)
        if os.path.isfile('junc.npz'):
            bb1 = np.load('junc.npz')
            length = bb1['length']
            orientxy = bb1['orientxy']
            orientxz = bb1['orientxz']
            orientyz = bb1['orientyz']
            orientxy, orientxz, orientyz, length = np.array(orientxy), np.array(orientxz), np.array(orientyz), np.array(length)
            orientxy, orientxz, orientyz, length = orientxy.astype(float), orientxz.astype(float), orientyz.astype(float), length.astype(float)
        else:
            orientxy = OrientationXY(segmentation)
            orientxz = OrientationXZ(segmentation)
            orientyz = OrientationYZ(segmentation)
            length = LengthJunc(segmentation)
            np.savez_compressed('junc.npz', orientxy=orientxy, orientxz=orientxz, orientyz=orientyz, length=length)
        os.chdir(cud)

        plt2=pn.pane.VTKVolume(length, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt3=pn.pane.VTKVolume(orientxy, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt4=pn.pane.VTKVolume(orientxz, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt5=pn.pane.VTKVolume(orientyz, sizing_mode='stretch_width', width=300, height=300, 
                                   spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt2.jslink(plt3, camera='camera', bidirectional=True)
        plt3.jslink(plt4, camera='camera', bidirectional=True)
        plt4.jslink(plt5, camera='camera', bidirectional=True)
        plt5.jslink(plt2, camera='camera', bidirectional=True)

        return pn.Column(pn.widgets.DataFrame(tablo_edg, name='DataFrame'),
                         pn.Row(plt3, plt4, plt5),
                         pn.Row(plt2)
                        )
    else:
        textindic.object = '## Alert\n' + ' You need skeleton file and original file'
        textindic.alert_type = 'danger'
        faceid = np.zeros((10,10,10))
        data = []
        tablo2 = pd.DataFrame(data, columns=['A', 'B', 'C'])
        plt3=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=225, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt4,plt5 = plt3,plt3
        return pn.Column(pn.widgets.DataFrame(tablo2, name='DataFrame'),
                         pn.Row(plt3, plt4, plt5)
                        )
    


## Apical analysis function

In [9]:
def apicalsurf(event):
    import os
    from skimage import io
    from basemodule import (junction_intensity, face_intensity, Junct_analy, Enlarge_fa)
    import pickle
    
    if (len(selfil3.value)>=1):
        fileother = selfil.value[0]
    else:
        fileother = ""
    if (len(selfil.value)>=2):
        if selfil.value[0][-5:]=='NDskl':
            fileskel = selfil.value[0]
            originalfil = selfil.value[1]
        else:
            originalfil = selfil.value[0]
            fileskel = selfil.value[1]
    
        workingdir = os.path.dirname(fileskel)
        file = os.path.basename(fileskel)
        cud, newd = os.getcwd(), file[:-5]
        os.chdir(workingdir)
        os.chdir('Arrays_' + newd)
        with open('segmentation.pkl', 'rb') as inp:
            segmentation = pickle.load(inp)
            
        voxelX0,voxelY0,voxelZ0,thick0,juncdil0 = voxelX.value,voxelY.value,voxelZ.value,thick.value,juncdil.value
        os.chdir(cud)
        #imot = imread(fileother)
        #imot = imot.astype(float)
        im = io.imread(os.path.join(workingdir,originalfil))
        im = im.astype(float)
        #tiff_junction = io.imread(os.path.join(directory,IMAGE_NAME))
        #junction_intensity(fileother, segmentation, int(juncdil0), tiffother.value)
        print((int(juncdil0),tiffother.value))
        segmentation = Junct_analy(segmentation,workingdir,originalfil, 3)
        segmentation, enlarge_face = Enlarge_fa(segmentation,workingdir,originalfil,float(thick0), 3)
        #junction_intensity(im, segmentation, int(juncdil0), tiffother.value)
        #enlarge_face = face_intensity(im, segmentation, float(thick0), int(juncdil0), tiffother.value)
        enlf = np.array(enlarge_face)
        enlf = enlf.astype(float)
    
        plt1=pn.pane.VTKVolume(im, sizing_mode='stretch_width', width=300, height=300, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
        plt2=pn.pane.VTKVolume(im, sizing_mode='stretch_width', width=300, height=300, 
                                       spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                                       edge_gradient=0, sampling=0)
        plt3=pn.pane.VTKVolume(enlf, sizing_mode='stretch_width', width=300, height=300,
                               spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                               edge_gradient=0, sampling=0)
        plt1.jslink(plt2, camera='camera', bidirectional=True)
        plt2.jslink(plt3, camera='camera', bidirectional=True)
        plt3.jslink(plt1, camera='camera', bidirectional=True)
    else:
        faceid = np.zeros((10,10,10))
        plt3=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=225, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt1,plt2 = plt3,plt3
    
    return pn.Column(pn.Row(plt1, plt2, plt3),)
    

## Cross analysis function

In [10]:
def crossanal(event):
    import os
    from basemodule import (PrepaCross, Max_Degre_Im, Nb_Node_Im, Mean_Connect_Im, Sum_Degre_Im, Mean_Degre_Im)
    from basemodule import (Sum_Betweenness_Im, Sum_Closenness_Im, Mean_Closenness_Im, Mean_Betweenness_Im)
    import pickle
    
    voxelX0,voxelY0,voxelZ0 = voxelX2.value,voxelY2.value,voxelZ2.value
    
    if (len(selfil.value)>=2):
        if selfil.value[0][-5:]=='NDskl':
            fileskel = selfil.value[0]
            originalfil = selfil.value[1]
        else:
            originalfil = selfil.value[0]
            fileskel = selfil.value[1]
    else:
        faceid = np.zeros((10,10,10))
        plt3=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=225, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt1,plt2 = plt3,plt3
        return pn.Column(pn.Row(plt1, plt2, plt3),)
    
    if (len(selfil2.value)>=2):
        if selfil2.value[0][-5:]=='NDskl':
            fileskel2 = selfil2.value[0]
            originalfil2 = selfil2.value[1]
        else:
            originalfil2 = selfil2.value[0]
            fileskel2 = selfil2.value[1]
    else:
        faceid = np.zeros((10,10,10))
        plt3=pn.pane.VTKVolume(faceid, sizing_mode='stretch_width', width=225, height=300, 
                                   spacing=(float(1),float(1),float(1)), interpolation='nearest', 
                                   edge_gradient=0, sampling=0)
        plt1,plt2 = plt3,plt3
        return pn.Column(pn.Row(plt1, plt2, plt3),)
    
    workingdir2 = os.path.dirname(fileskel2)
    workingdir = os.path.dirname(fileskel)
    file = os.path.basename(fileskel)
    cud, newd = os.getcwd(), file[:-5]
    os.chdir(workingdir)
    os.chdir('Arrays_' + newd)
    with open('segmentation4cross.pkl', 'rb') as inp:
        segmentation = pickle.load(inp)
    os.chdir(cud)
    file2 = os.path.basename(fileskel2)
    cud, newd2 = os.getcwd(), file2[:-5]
    
    os.chdir(workingdir2)
    os.chdir('Arrays_' + newd2)
    textindic4.object = '## Alert\n' + ' Network already done  ...wait'
    if os.path.isfile('cross.npz'):
        bb1 = np.load('cross.npz')
        max_degree_image = np.array(bb1['max_degree_image']).astype(float)
        nb_node_pcell = np.array(bb1['nb_node_pcell']).astype(float)
        connection = np.array(bb1['connection']).astype(float)
        sum_degree = np.array(bb1['sum_degree']).astype(float)
        sum_between = np.array(bb1['sum_between']).astype(float)
        sum_closen = np.array(bb1['sum_closen']).astype(float)
        mean_closen = np.array(bb1['mean_closen']).astype(float)
        mean_between = np.array(bb1['mean_between']).astype(float)
        mean_degre = np.array(bb1['mean_degre']).astype(float)
        
    else:
        segmentation, node_df, G = PrepaCross(workingdir2, fileskel2, segmentation, float(voxelX0), float(voxelY0), float(voxelZ0))
        max_degree_image = Max_Degre_Im(workingdir, originalfil, node_df, G, segmentation)
        nb_node_pcell = Nb_Node_Im(workingdir, originalfil, node_df, segmentation)
        connection = Mean_Connect_Im(workingdir, originalfil, node_df, segmentation)
        sum_degree = Sum_Degre_Im(workingdir, originalfil, node_df, G, segmentation)
        sum_between = Sum_Betweenness_Im(workingdir, originalfil, node_df, G, segmentation)
        sum_closen = Sum_Closenness_Im(workingdir, originalfil, node_df, G, segmentation)
        mean_closen = Mean_Closenness_Im(workingdir, originalfil, node_df, G, segmentation)
        mean_between = Mean_Betweenness_Im(workingdir, originalfil, node_df, G, segmentation)
        mean_degre = Mean_Degre_Im(workingdir, originalfil, node_df, G, segmentation)
        np.savez_compressed('cross.npz', max_degree_image=max_degree_image, nb_node_pcell=nb_node_pcell,
                            connection=connection, sum_degree=sum_degree, sum_between=sum_between, sum_closen=sum_closen,
                           mean_closen=mean_closen, mean_between=mean_between, mean_degre=mean_degre)
    
    os.chdir(cud)
    textindic4.object = '## Alert\n' + ' Beginning plot'
    plt0=pn.pane.VTKVolume(max_degree_image, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt1=pn.pane.VTKVolume(nb_node_pcell, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt2=pn.pane.VTKVolume(connection, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt3=pn.pane.VTKVolume(sum_degree, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt4=pn.pane.VTKVolume(sum_between, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt5=pn.pane.VTKVolume(sum_closen, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt6=pn.pane.VTKVolume(mean_closen, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt7=pn.pane.VTKVolume(mean_between, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    plt8=pn.pane.VTKVolume(mean_degre, sizing_mode='stretch_width', width=300, height=600, 
                        spacing=(float(voxelZ0),float(voxelY0),float(voxelX0)), interpolation='nearest', 
                        edge_gradient=0, sampling=0)
    
    plt0.jslink(plt1, camera='camera', bidirectional=True)
    plt1.jslink(plt2, camera='camera', bidirectional=True)
    plt2.jslink(plt3, camera='camera', bidirectional=True)
    plt3.jslink(plt4, camera='camera', bidirectional=True)
    plt4.jslink(plt5, camera='camera', bidirectional=True)
    plt5.jslink(plt6, camera='camera', bidirectional=True)
    plt6.jslink(plt7, camera='camera', bidirectional=True)
    plt7.jslink(plt8, camera='camera', bidirectional=True)
    plt8.jslink(plt0, camera='camera', bidirectional=True)

    return pn.Column(pn.Row(btn_max_degree_image,btn_nb_node_pcell,btn_connection),
                     pn.Row(plt0, plt1, plt2),
                     pn.Row(btn_sum_degree,btn_sum_between,btn_sum_closen),
                     pn.Row(plt3, plt4, plt5),
                     pn.Row(btn_mean_closen,btn_mean_between,btn_mean_degre),
                     pn.Row(plt6, plt7, plt8),
                        )


In [11]:

dissect = pn.Column(app_bar,
    '# **<div align="center"> 1 - Segmentation</div>**',
    pn.layout.Divider(margin=(-20, 0, 0, 0)),
    '### **<div align="center">Select NDskl file and Tiff file</div>**',
    selfil,
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    pn.Row(voxelX,voxelY,voxelZ,thick,juncdil),
    pn.layout.Divider(margin=(-20, 0, 0, 0)),
    '### **<div align="center">1-1 - Raw Skeleton and Raw Image </div>**',
    pn.Row(textindic0, buttonskelontif),
    pn.Row(buttonsrawim,buttonraskel,buttonimskel),
    pn.panel(pn.bind(binontiff, buttonskelontif), loading_indicator=True),
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    '### **<div align="center">1-2 - Skeleton and Vertices, Junction, Faces </div>**',
    pn.Row(textindic, buttonstartclean),
    pn.panel(pn.bind(cleantask, buttonstartclean), loading_indicator=True),
    pn.Row(buttonskelvert,buttonjuncid,buttoncellid),
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    '# **<div align="center"> 2 - Morphological Analysis</div>**',
    pn.layout.Divider(margin=(-20, 0, 0, 0)),
    '### **<div align="center">Select morphological parameters</div>**',
    pn.Row(multi_choice),      
    pn.panel(pn.bind(morphoana, buttonmorphoanaly), loading_indicator=True),
    pn.Row(buttonmorphoanaly),
    pn.panel(pn.bind(juncana, buttonjuncanaly), loading_indicator=True),
    pn.Row(buttonjuncanaly),
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    '# **<div align="center"> 3 - APICAL Surface Intensity</div>**',
    pn.layout.Divider(margin=(-20, 0, 0, 0)),
    selfil3,
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    pn.Row(textindic3,tiffother,buttontiffother),
    pn.panel(pn.bind(apicalsurf, buttontiffother), loading_indicator=True),
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    '# **<div align="center"> 4 - Network Analysis</div>**',
    pn.layout.Divider(margin=(-20, 0, 0, 0)),
    selfil2,
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    pn.Row(voxelX2,voxelY2,voxelZ2),
    pn.Row(textindic2, buttonstartnet),
    pn.panel(pn.bind(networktask, buttonstartnet), loading_indicator=True),
    pn.layout.Divider(margin=(20, 0, 0, 0)),
    '# **<div align="center"> 5 - Cross Analysis</div>**',
    pn.layout.Divider(margin=(-20, 0, 0, 0)),
    pn.Row(textindic4, buttoncross),
    pn.panel(pn.bind(crossanal, buttoncross), loading_indicator=True),
)

In [12]:
dissect.show();

Launching server at http://localhost:40395
